# Hazard map

In [76]:
#### version 2

import rasterio
import numpy as np



base_folder = 'N:/C2205_FAIRiCUBE/f02_data/d060_data_LUXEMBOURG/f01_Lux_neopyhtes/f02_luxembourg_data/f07_hazard_map'
distribution_raster = "N:/C2205_FAIRiCUBE/f02_data/d060_data_LUXEMBOURG/f01_Lux_neopyhtes/f02_luxembourg_data/f05_distribution/f02_distribution_model_luxembourg/heracleum_mantegazzianum_RandomForest_model_v2.tif"




# Function to apply a buffer to a raster and write the buffer distance as the value
def buffer_raster(input_raster, output_raster, buffer_distance):
    # Open the raster file
    with rasterio.open(input_raster) as src:
        # Read the raster data
        data = src.read(1)  # Read the first band (usually the data you want)
        
        # Get raster metadata
        profile = src.profile
        
        # Get pixel size (assuming square cells)
        pixel_size = abs(profile['transform'][0])  # cell size in map units (e.g., meters)
        
        # Buffer distance in pixels
        buffer_pixels = int(buffer_distance / pixel_size)
        
        # Create a kernel (a square window) to apply around each pixel
        kernel_size = 2 * buffer_pixels + 1
        kernel = np.ones((kernel_size, kernel_size), dtype=np.uint8)
        
        # Create an output array for the buffered raster
        buffered_data = np.zeros_like(data, dtype=np.float32)
        
        # Apply the kernel as a convolution (simple box filter)
        for i in range(buffer_pixels, data.shape[0] - buffer_pixels):
            for j in range(buffer_pixels, data.shape[1] - buffer_pixels):
                # Define the region of interest (ROI) around the current pixel
                roi = data[i-buffer_pixels:i+buffer_pixels+1, j-buffer_pixels:j+buffer_pixels+1]
                
                # If any of the ROI values are non-zero (indicating presence), set the pixel as the buffer distance
                if np.any(roi > 0):  # Adjust this condition for more complex buffering
                    buffered_data[i, j] = buffer_distance  # Assign the buffer distance as the pixel value
        
        # Write the buffered raster to an output file
        profile.update(dtype=rasterio.float32, count=1)
        with rasterio.open(output_raster, 'w', **profile) as dst:
            dst.write(buffered_data, 1)  # Write the first band of the buffered raster

# Example usage





In [ ]:

buffer_list = (10
            ,20
            ,30
            ,40
            ,50
            ,60
            ,70
            ,80
            ,90
            ,100
            ,110
            ,120
            ,130
            ,140
            ,150
            ,160
            ,170
            ,180
            ,190
            ,200
            ,210
            ,220
            ,230
            ,240
            ,250
            ,260
            ,270
            ,280
            ,290
            ,300
            ,310
            ,320
            ,330
            ,340
            ,350
            ,360
            ,370
            ,380
            ,390
            ,400
            ,410
            ,420
            ,430
            ,440
            ,450
            ,460
            ,470
            ,480
            ,490
            ,500
            ,510
            ,520
            ,530
            ,540
            ,550
            ,560
            ,570
            ,580
            ,590
            ,600
            )

for buf in buffer_list:
    
    input_raster = distribution_raster  
    buffer_distance = buf  
    output_raster = base_folder +"/"+ 'single_heracleum_mantegazzianum'+"/"+'tmp_buffer_'+str(buffer_distance)+"_heracl.tif"
    buffer_raster(input_raster, output_raster, buffer_distance)




In [78]:
# read and set up raster list
import glob
import os
import re

# Define the folder path
folder_path = base_folder +"/"+ 'single_heracleum_mantegazzianum'

# Create a list of all .tif files in the folder
raster_list = glob.glob(os.path.join(folder_path, "*.tif"))


# Extract the numeric part (from the "tmp_buffer_x_heracl" part) and sort  # new order
# Sort the list of rasters in descending order by the number in the filename
raster_list = sorted(raster_list, key=lambda x: int(re.search(r'(\d+)_heracl', x).group(1)), reverse=True)

# Print the sorted list
for raster in raster_list:
    print(raster)



N:/C2205_FAIRiCUBE/f02_data/d060_data_LUXEMBOURG/f01_Lux_neopyhtes/f02_luxembourg_data/f07_hazard_map/single_heracleum_mantegazzianum\tmp_buffer_600_heracl.tif
N:/C2205_FAIRiCUBE/f02_data/d060_data_LUXEMBOURG/f01_Lux_neopyhtes/f02_luxembourg_data/f07_hazard_map/single_heracleum_mantegazzianum\tmp_buffer_590_heracl.tif
N:/C2205_FAIRiCUBE/f02_data/d060_data_LUXEMBOURG/f01_Lux_neopyhtes/f02_luxembourg_data/f07_hazard_map/single_heracleum_mantegazzianum\tmp_buffer_580_heracl.tif
N:/C2205_FAIRiCUBE/f02_data/d060_data_LUXEMBOURG/f01_Lux_neopyhtes/f02_luxembourg_data/f07_hazard_map/single_heracleum_mantegazzianum\tmp_buffer_570_heracl.tif
N:/C2205_FAIRiCUBE/f02_data/d060_data_LUXEMBOURG/f01_Lux_neopyhtes/f02_luxembourg_data/f07_hazard_map/single_heracleum_mantegazzianum\tmp_buffer_560_heracl.tif
N:/C2205_FAIRiCUBE/f02_data/d060_data_LUXEMBOURG/f01_Lux_neopyhtes/f02_luxembourg_data/f07_hazard_map/single_heracleum_mantegazzianum\tmp_buffer_550_heracl.tif
N:/C2205_FAIRiCUBE/f02_data/d060_data_LU

In [79]:
import rasterio
from rasterio.crs import CRS
import numpy as np

def overlay_rasters(raster_list, output_raster):
    """
    Overlay multiple buffer rasters and generate a final raster where each pixel contains the lowest buffer distance.
    
    Args:
        raster_list (list): List of file paths to input raster files.
        output_raster (str): File path to save the resulting raster.
    """
    # Open the first raster to use as a template
    with rasterio.open(raster_list[0]) as src:
        profile = src.profile
        profile.update(dtype=rasterio.float32)  # Ensure the output is float32
        # Read the first raster data
        combined_data = src.read(1).astype(np.float32)
    
    # Initialize the combined raster with high values (to find the minimum distance)
    combined_data[combined_data > 0] = np.inf  # Set initial non-zero buffer values to infinity
    
    # Iterate over each raster in the list
    for raster_path in raster_list:
        with rasterio.open(raster_path) as src:
            # Read current raster data
            data = src.read(1).astype(np.float32)
            # Combine using minimum values (ignoring zero background)
            combined_data = np.minimum(combined_data, np.where(data > 0, data, np.inf))
    
    # Replace infinity with zero for non-buffered areas
    combined_data[np.isinf(combined_data)] = 0
   

    # Write the resulting raster to disk
    with rasterio.open(output_raster, 'w', **profile) as dst:
        dst.write(combined_data, 1)

# Example Usage
raster_list = raster_list

output_raster = base_folder +"/"+"heracleum_mantegazzianum_buffer_distance.tif"

overlay_rasters(raster_list, output_raster)




In [80]:
import rasterio


# Open the raster file
with rasterio.open(output_raster) as src:
    crs = src.crs  # Access the CRS

# Print the CRS
print("CRS:", crs)

CRS: None


# update crs - using gdal wrap




In [41]:
#gdalwarp -s_srs EPSG:2169 -t_srs EPSG:2169 -r near -of GTiff -co COMPRESS=DEFLATE -co PREDICTOR=2 -co ZLEVEL=9 N:/C2205_FAIRiCUBE/f02_data/d060_data_LUXEMBOURG/f01_Lux_neopyhtes/f02_luxembourg_data/test/hazard_map/final_buffer_distance.tif C:/Users/ManuelLöhnertz/AppData/Local/Temp/processing_PozwJp/76a3e206b5c54faaa83f46f01e6811a2/OUTPUT.tif

In [88]:

## not working:
from osgeo import gdal

# Input and output file paths
input_file = output_raster
output_file = output_raster[:-4]+"_2169.tif"


luref_projection_epsg2169 ="+proj=lcc +lat_1=49.8333339 +lat_2=51.16666723333333 +lat_0=49.0 +lon_0=6.166666666666667 +x_0=80000 +y_0=100000 +ellps=GRS80 +units=m +no_defs"

#luref_projection_epsg2169 ="EPSG:2169","+proj=tmerc +lat_0=49.8333333333333 +lon_0=6.16666666666667 +k=1 +x_0=80000 +y_0=100000 +ellps=intl +towgs84=-265.8867,76.9851,20.2667,0.33746,3.09264,-2.53861,0.4598 +units=m +no_defs +type=crs");
# Define warp options
warp_options = gdal.WarpOptions(
    srcSRS=luref_projection_epsg2169,  # Source CRS
    dstSRS=luref_projection_epsg2169,  # Target CRS
    resampleAlg="nearest",  # Resampling method (nearest neighbor)
    format="GTiff"  # Output format
    )

# Perform the warp operation
gdal.Warp(destNameOrDestDS=output_file, srcDSOrSrcDSTab=input_file, options=warp_options)


In [89]:
# Open the raster file
with rasterio.open(output_file) as src:
    crs = src.crs  # Access the CRS

# Print the CRS
print("CRS:", crs)

CRS: PROJCS["unknown",GEOGCS["unknown",DATUM["Unknown_based_on_GRS80_ellipsoid",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",49],PARAMETER["central_meridian",6.16666666666667],PARAMETER["standard_parallel_1",49.8333339],PARAMETER["standard_parallel_2",51.1666672333333],PARAMETER["false_easting",80000],PARAMETER["false_northing",100000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


## overlay buffer raster with poi raster

Problems with exetent and crs.. save output_file in qgis witht 2169 proj and to aoi-extent Extent	72850.0000000000000000,69590.0000000000000000 : 82570.0000000000000000,80030.0000000000000000

In [97]:
import rasterio
import numpy as np

# Input raster files
input_raster = r"N:\C2205_FAIRiCUBE\f02_data\d060_data_LUXEMBOURG\f01_Lux_neopyhtes\f02_luxembourg_data\f07_hazard_map\heracleum_mantegazzianum_buffer_distance_2169v2.tif"
mask_raster =  r'N:\C2205_FAIRiCUBE\f02_data\d060_data_LUXEMBOURG\f01_Lux_neopyhtes\f02_luxembourg_data\f02_raster_10m_collection\aoi_osm_2024_10m_b4.tif'




band_list_aoi = (1,2,3,4)

for band in band_list_aoi:
    output_raster =  r"N:\C2205_FAIRiCUBE\f02_data\d060_data_LUXEMBOURG\f01_Lux_neopyhtes\f02_luxembourg_data\f07_hazard_map\heracleum_mantegazzianum_hazard_band"+str(band)+".tif"

    # Open the input raster and mask raster
    with rasterio.open(input_raster) as src, rasterio.open(mask_raster) as mask_src:
        
        # Read the data from both rasters
        input_data = src.read(1)  # Read the first band of the input raster
        mask_data = mask_src.read(band)  # Read the first band of the mask raster
        
        # Set the NoData value for the input raster
        nodata_value = src.nodata
        
        # Apply the mask: We will keep the input raster values only where the mask has a non-zero value
        # Set values outside the mask to NoData
        masked_data = np.where(mask_data > 0, input_data, nodata_value)

        # Update metadata for the output raster
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "count": 1,  # single band raster
            "nodata": nodata_value
        })
        
        # Write the masked data to the output file
        with rasterio.open(output_raster, 'w', **out_meta) as dst:
            dst.write(masked_data, 1)

